# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


from sqlalchemy import create_engine

In [2]:
# import libraries nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jinyanxiang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jinyanxiang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jinyanxiang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# import libraries sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report, log_loss, f1_score
from sklearn.model_selection import GridSearchCV

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessageData', engine)


X = df['message']
y = df[df.columns[4:]]

In [5]:
#check frequencies of each category
FreqTb = pd.DataFrame()
for col in y.columns:
    tb = pd.DataFrame(y[col].value_counts())
    
    FreqTb = pd.concat([FreqTb, tb], axis = 1)

In [6]:
#frequency table part 1
FreqTb[FreqTb.columns[:18]]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death
0,6116,21716.0,26062.0,15339.0,24099.0,24869.0,25456.0,25709.0,25321.0,26180.0,24511.0,23263.0,23872.0,25776.0,25577.0,25882.0,25306.0,24988.0
1,19876,4464.0,118.0,10841.0,2081.0,1311.0,724.0,471.0,859.0,NaN,1669.0,2917.0,2308.0,404.0,603.0,298.0,874.0,1192.0
2,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#frequency table part 2
FreqTb[FreqTb.columns[19:]]

,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,24475.0,24981.0,24849.0,25648.0,26021.0,25897.0,26060.0,25871.0,25029.0,18894.0,24031.0,23740.0,25898.0,23728.0,25652.0,24804.0,21116.0
1,1705.0,1199.0,1331.0,532.0,159.0,283.0,120.0,309.0,1151.0,7286.0,2149.0,2440.0,282.0,2452.0,528.0,1376.0,5064.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#based on the frequency table, most categories are highly skewed and one category "child-alone" only has 0 labels
#drop the 188 label 2s which could possibly be record error (as other categories do not have 2 and the meaning of 2 is unclear)
df = df[df.related != 2]

In [9]:
#drop the class - child_alone, which has not relevant cases in the sample (i.e., all zeros)
df.drop('child_alone', axis = 1, inplace = True)

In [10]:
X = df['message']
y = df[df.columns[4:]]

In [11]:
X.shape, y.shape

((25992,), (25992, 35))

In [213]:
X.ndim

1

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    
    token = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    cleaned_tokens = []
    
    for tok in token:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        cleaned_tokens.append(clean_tok)
    
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline_LR = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    
    ('cfl', MultiOutputClassifier(LogisticRegression())) #use LogisticRegression as the classifier
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
X_train.shape, y_train.shape

((20793,), (20793, 35))

In [16]:
pipeline_LR.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                ('tfidf', TfidfTransformer()),
                ('cfl', MultiOutputClassifier(estimator=LogisticRegression()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_lr = pipeline_LR.predict(X_test)

In [18]:
y_true_lr = np.array(y_test.values)

In [19]:
def ClassificationReport(y_true, y_pred, classifier):
    print("the classification report of Multi-label classifiers with ", classifier, " estimator")
    
    print("\n")
    
    print(classification_report(y_true, y_pred,target_names = y.columns))

In [20]:
ClassificationReport(y_true_lr, y_pred_lr, "Logistic Regression")

the classification report of Multi-label classifiers with  Logistic Regression  estimator


                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      3969
               request       0.85      0.58      0.69       917
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.69      0.73      2199
          medical_help       0.65      0.17      0.26       427
      medical_products       0.78      0.22      0.34       246
     search_and_rescue       0.80      0.06      0.11       138
              security       0.00      0.00      0.00        89
              military       0.64      0.10      0.17       160
                 water       0.83      0.48      0.61       334
                  food       0.88      0.59      0.70       630
               shelter       0.87      0.43      0.57       500
              clothing       0.90      0.28      0.43        68
           

In [21]:
#based on the classifciation report, the performance of the algorithm varies by the predicated class 
#it performs worst when the message category has highly skewed data (e.g., offer, only 118 1s vs. 26062 0s in the original df)

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
#get the list of parameters that can be tuned
pipeline_LR.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'cfl', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'cfl__estimator__C', 'cfl__estimator__class_weight', 'cfl__estimator__dual', 'cfl__estimator__fit_intercept', 'cfl__estimator__intercept_scaling', 'cfl__estimator__l1_ratio', 'cfl__estimator__max_iter', 'cfl__estimator__multi_class', 'cfl__estimator__n_jobs', 'cfl__estimator__penalty', 'cfl__estimator__random_state', 'cfl__estimator__solver', 'cfl__estimator__tol', 'cfl__estimator__verbose', 'cfl__estimator__warm_start', 'cfl__estimator', 'cfl__n_jobs'])

In [23]:
parameters = {
    'cfl__estimator__C': np.arange(0,1.2,0.2), 
    #tune the regularization strength, default is 1
    'cfl__estimator__class_weight':['balanced'], 
    #set the class_weight to be "balanced" as most message labels have unbalanced classes, 
    #default is None, which treated all classes equally
    'cfl__estimator__max_iter': [100, 500, 1000],
    #increase the max iterations, default is 100
    'cfl__estimator__n_jobs':[-1]
    #set n_jos to be -1 to use all processors
    }


cv = GridSearchCV(pipeline_LR, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
#set scoring to be f1_micro because the classes have unbalanced weights
#set n_jobs to be -1 to use all processors

cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('cfl',
                                        MultiOutputClassifier(estimator=LogisticRegression()))]),
             n_jobs=-1,
             param_grid={'cfl__estimator__C': array([0. , 0.2, 0.4, 0.6, 0.8, 1. ]),
                         'cfl__estimator__class_weight': ['balanced'],
                         'cfl__estimator__max_iter': [100, 500, 1000],
                         'cfl__estimator__n_jobs': [-1]},
             scoring='f1_micro')

In [24]:
cv.best_params_

{'cfl__estimator__C': 1.0,
 'cfl__estimator__class_weight': 'balanced',
 'cfl__estimator__max_iter': 100,
 'cfl__estimator__n_jobs': -1}

In [65]:
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                ('tfidf', TfidfTransformer()),
                ('cfl',
                 MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                                    n_jobs=-1)))])

In [142]:
cv.best_score_

0.6267770102436501

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
y_pred_lr_cv = cv.predict(X_test)
y_true_lr_cv = y_test.values

ClassificationReport(y_true_lr_cv, y_pred_lr_cv, "Logistic Regression (balanced weight)")

the classification report of Multi-label classifiers with  Logistic Regression (balanced weight)  estimator


                        precision    recall  f1-score   support

               related       0.92      0.83      0.87      3969
               request       0.63      0.81      0.71       917
                 offer       0.08      0.21      0.12        28
           aid_related       0.73      0.77      0.75      2199
          medical_help       0.35      0.64      0.45       427
      medical_products       0.30      0.64      0.41       246
     search_and_rescue       0.14      0.36      0.20       138
              security       0.11      0.33      0.17        89
              military       0.35      0.74      0.47       160
                 water       0.56      0.87      0.68       334
                  food       0.70      0.83      0.76       630
               shelter       0.56      0.82      0.67       500
              clothing       0.31      0.68      0.42    

In [27]:
#the performance of the model with balanced weights assigned to each category improve marginally
#the improvement is most pronounced for message categories with severe skewness issues

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### 8.1. Improving the Cross-Validated Logistic Regresion

In [30]:
#improvement 1: try different threshold for classification (default = 0.5)
def ChangeThreshold(y_true, y_pred_proba, thresholds):

    Threshold_Results = []
    
    for th in thresholds:
        y_pred_new_threshold = np.zeros(y_true.shape)
        
        for i, predicts in enumerate(y_pred_proba):
            y_pred_new_threshold_i = (predicts[:,1] >= th).astype(int)
            y_pred_new_threshold[:,i] = y_pred_new_threshold_i 
        
        f1_micro = f1_score(y_true, y_pred_new_threshold, average = 'micro')    
        f1_weighted = f1_score(y_true, y_pred_new_threshold, average = 'weighted')

        
        Threshold_Results.append(("threshold: {}".format(th.round(2)), 
                                  "f1_micro: {}".format(f1_micro.round(3)),
                                  "f1_weighted: {}".format(f1_weighted.round(3))
                                 ))
        
    return Threshold_Results
    

In [31]:
y_pred_proba = cv.predict_proba(X_test)

#threshold to be tested
thresholds = np.arange(0.2,0.8,0.1)

ChangeThreshold(y_true_lr_cv, y_pred_proba, thresholds)

[('threshold: 0.2', 'f1_micro: 0.393', 'f1_weighted: 0.533'),
 ('threshold: 0.3', 'f1_micro: 0.502', 'f1_weighted: 0.601'),
 ('threshold: 0.4', 'f1_micro: 0.584', 'f1_weighted: 0.648'),
 ('threshold: 0.5', 'f1_micro: 0.635', 'f1_weighted: 0.673'),
 ('threshold: 0.6', 'f1_micro: 0.654', 'f1_weighted: 0.673'),
 ('threshold: 0.7', 'f1_micro: 0.638', 'f1_weighted: 0.642'),
 ('threshold: 0.8', 'f1_micro: 0.578', 'f1_weighted: 0.572')]

In [32]:
#it suggests that slightly increase the threshold to 0.6 from 0.5 may slighly improve the f1_score

#### 8.2. Try Random Forest Classifier

In [33]:
#improvement 2a: try different classifier models (random forest classifier)
pipeline_RF = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    
    ('cfl', MultiOutputClassifier(RandomForestClassifier(class_weight = 'balanced', n_jobs = -1))) #use LogisticRegression as the classifier
])

In [34]:
pipeline_RF.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                ('tfidf', TfidfTransformer()),
                ('cfl',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        n_jobs=-1)))])

In [35]:
y_true_rf = y_test.values
y_pred_rf = pipeline_RF.predict(X_test)

In [36]:
ClassificationReport(y_true_rf, y_pred_rf, "Random Forest Classification")

the classification report of Multi-label classifiers with  Random Forest Classification  estimator


                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      3969
               request       0.90      0.44      0.60       917
                 offer       0.00      0.00      0.00        28
           aid_related       0.79      0.63      0.70      2199
          medical_help       0.65      0.04      0.07       427
      medical_products       0.92      0.05      0.09       246
     search_and_rescue       0.50      0.01      0.01       138
              security       0.00      0.00      0.00        89
              military       0.00      0.00      0.00       160
                 water       0.92      0.14      0.24       334
                  food       0.93      0.27      0.42       630
               shelter       0.93      0.13      0.23       500
              clothing       1.00      0.04      0.08        68
  

#### 8.3. Try Support Vector Machine

In [37]:
#improvement 3b: try different classifier models (SVM)
pipeline_SVM = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    
    ('cfl', MultiOutputClassifier(SVC(class_weight = 'balanced'))) #use LogisticRegression as the classifier
])

In [38]:
pipeline_SVM.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                ('tfidf', TfidfTransformer()),
                ('cfl',
                 MultiOutputClassifier(estimator=SVC(class_weight='balanced')))])

In [41]:
y_true_svm = y_test.values
y_pred_svm = pipeline_SVM.predict(X_test)

In [40]:
ClassificationReport(y_true_svm, y_pred_svm, "Support Vector Machine (Classifier)")

the classification report of Multi-label classifiers with  Support Vector Machine (Classifier)  estimator


                        precision    recall  f1-score   support

               related       0.89      0.89      0.89      3969
               request       0.75      0.72      0.74       917
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.75      0.75      2199
          medical_help       0.60      0.37      0.45       427
      medical_products       0.65      0.38      0.48       246
     search_and_rescue       0.61      0.14      0.22       138
              security       0.00      0.00      0.00        89
              military       0.60      0.33      0.42       160
                 water       0.71      0.77      0.74       334
                  food       0.82      0.76      0.79       630
               shelter       0.77      0.64      0.70       500
              clothing       0.78      0.47      0.59      

#### 8.4 Boost the Logistic Regression model using Adaptive Boosting

In [53]:
#use adaptive boosting method to improve the performance of logistic regression
#use AdaBoostClassifier to boost the logistic regression 
pipeline_boosted = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('cfl', MultiOutputClassifier(AdaBoostClassifier(base_estimator = LogisticRegression(class_weight = 'balanced', n_jobs = -1))))
])

In [54]:
pipeline_boosted.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fab88892430>)),
                ('tfidf', TfidfTransformer()),
                ('cfl',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=LogisticRegression(class_weight='balanced',
                                                                                                      n_jobs=-1))))])

In [55]:
y_true_boosted = y_test.values
y_pred_boosted = pipeline_boosted.predict(X_test)

In [57]:
ClassificationReport(y_true_boosted, y_pred_boosted, "Logistic Regression (balanced weight)")

the classification report of Multi-label classifiers with  Logistic Regression (balanced weight)  estimator


                        precision    recall  f1-score   support

               related       0.85      0.79      0.82      3969
               request       0.56      0.73      0.64       917
                 offer       0.03      0.39      0.06        28
           aid_related       0.65      0.70      0.67      2199
          medical_help       0.18      0.73      0.28       427
      medical_products       0.15      0.71      0.25       246
     search_and_rescue       0.08      0.59      0.13       138
              security       0.03      0.58      0.06        89
              military       0.10      0.86      0.18       160
                 water       0.39      0.84      0.53       334
                  food       0.53      0.72      0.61       630
               shelter       0.35      0.71      0.47       500
              clothing       0.12      0.69      0.21    

### 9. Export your model as a pickle file

In [ ]:
#Conclusion: in terms of performance (metrics: F1-score), 
#SVM peforms best, followed by logistic regression, and random forest performs worst. 
#However, since SVM only performs slightly better on micro F1-score than logistic regression 
#but has higher computational cost, 
#I would adopt the cross-validated logistic regression as the optimized classifier.

In [131]:
threshold_report = ChangeThreshold(y_true_lr_cv, y_pred_proba,thresholds)

In [215]:
import pickle

with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [185]:
#check = pickle.load(open('MLclassifier.pkl','rb'))
#check

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.